In [ ]:
import xgboost as xgb
import pandas as pd
import numpy as np
import datetime

In [ ]:
df_pop = pd.read_csv('../../data/popular_data.csv')
df_pop

In [ ]:
df_pop["timestamp"]=df_pop["timestamp"].astype("datetime64",copy=True)

In [ ]:
import time
start = time.time()

df_pop['day_of_year'] = df_pop['timestamp'].dt.dayofyear
df_pop['day_of_month'] = df_pop['timestamp'].dt.day
df_pop['day_of_week'] = df_pop['timestamp'].dt.dayofweek
df_pop['month_of_year'] = df_pop['timestamp'].dt.month
df_pop['quarter_of_year'] = df_pop['timestamp'].dt.quarter
df_pop['is_quarter_start'] = df_pop['timestamp'].dt.is_quarter_start
df_pop['is_quarter_end'] = df_pop['timestamp'].dt.is_quarter_end
df_pop['is_month_start'] = df_pop['timestamp'].dt.is_month_start
df_pop['is_month_end'] = df_pop['timestamp'].dt.is_month_end

print("first block",time.time()-start)
tmp=time.time()
df_pop['days_since_last_change'] = df_pop["timestamp"]-df_pop.groupby(['infobox_key', 'property_name'])['timestamp'].shift(+1).fillna(pd.Timestamp('20990101'))
df_pop['days_since_last_change'] = df_pop['days_since_last_change'].apply(lambda x: round(pd.Timedelta(x).total_seconds()/(3600*24)))
df_pop['days_since_last_change'][df_pop['days_since_last_change']<0]=-1
print("days_since_last_change",time.time()-tmp)
tmp=time.time()
df_pop['days_since_last_2_changes'] = df_pop["timestamp"]-df_pop.groupby(['infobox_key', 'property_name'])['timestamp'].shift(+2).fillna(pd.Timestamp('20990101'))
df_pop['days_since_last_2_changes'] = df_pop['days_since_last_2_changes'].apply(lambda x: round(pd.Timedelta(x).total_seconds()/(3600*24)))
df_pop['days_since_last_2_changes'][df_pop['days_since_last_2_changes']<0]=-1
print("days_since_last_2_change",time.time()-tmp)
tmp=time.time()
df_pop['days_since_last_3_changes'] = df_pop["timestamp"]-df_pop.groupby(['infobox_key', 'property_name'])['timestamp'].shift(+3).fillna(pd.Timestamp('20990101'))
df_pop['days_since_last_3_changes'] = df_pop['days_since_last_3_changes'].apply(lambda x: round(pd.Timedelta(x).total_seconds()/(3600*24)))
df_pop['days_since_last_3_changes'][df_pop['days_since_last_3_changes']<0]=-1
print("days_since_last_3_change",time.time()-tmp)
tmp=time.time()

df_pop['days_until_next_change'] = df_pop.groupby(['infobox_key', 'property_name'])['days_since_last_change'].shift(-1)
df_pop['days_until_next_change'] = pd.to_numeric(df_pop['days_until_next_change'].fillna(-1),downcast="integer")
print("days_until_next_change",time.time()-tmp)
tmp=time.time()
df_pop['days_between_last_and_2nd_to_last_change'] = df_pop.groupby(['infobox_key', 'property_name'])['days_since_last_change'].shift(+1)
df_pop['days_between_last_and_2nd_to_last_change'] = pd.to_numeric(df_pop['days_between_last_and_2nd_to_last_change'].fillna(-1),downcast="integer")
print("days_between_last_and_2nd_to_last_change",time.time()-tmp)
tmp=time.time()
df_pop['mean_change_frequency_all_previous'] = df_pop.groupby(['infobox_key', 'property_name'])['days_since_last_change'].apply(lambda x: x.iloc[0:1].append(x.iloc[1:].expanding().mean()))
print("mean_change_frequency_all_previous",time.time()-tmp)
tmp=time.time()
df_pop['mean_change_frequency_last_3'] = df_pop.groupby(['infobox_key', 'property_name'])['days_since_last_change'].apply(lambda x: x.iloc[0:1].append(x.iloc[1:].rolling(3).mean())).fillna(-1)
print("mean_change_frequency_last_3",time.time()-tmp)

print("total", time.time()-start)

df_pop

In [ ]:
df_pop.head(20)